In [2]:
import pandas as pd
import numpy as np
import os


In [7]:

import glob
# This script processes CSV files containing data from a specific experiment.
# It reads the files, filters the data, adds cumulative volume, and formats the output.

# Ask user for directory containing the CSV files
directory = input("enter the directory for the titration files: ").strip()
reference_path = os.path.join(directory, "ref.01a")

# Load and preserve the .01a header
with open(reference_path, "r") as ref_file:
    header_lines = []
    for line in ref_file:
        header_lines.append(line)
        if line.strip().startswith("0,"):  # Stop after header block
            break

# Process each CSV file in the directory
csv_files = glob.glob(os.path.join(directory, "*.csv"))

for file_path in csv_files:
    if "filtered" in file_path or "formatted" in file_path:
        continue  # Skip already-processed files

    # Step 1: Read and transpose
    df = pd.read_csv(file_path, header=None)
    df_transposed = df.T.reset_index(drop=True)
    df_transposed.columns = ['mV', 'stable', 'dose']

    # Step 2: Filter for stable == 1
    df_filtered = df_transposed[df_transposed['stable'] == 1].copy()

    # Step 3: Add cumulative volume (0.1 steps)
    running_total = -0.1
    running_totals = []
    for _ in df_filtered.itertuples():
        running_total += 0.1
        running_totals.append(round(running_total, 1))

    df_filtered['volume'] = running_totals
    df_filtered.insert(0, 'volume', df_filtered.pop('volume'))

    # Step 4: Save filtered CSV
    base_name = os.path.splitext(os.path.basename(file_path))[0]
    filtered_path = os.path.join(directory, f"{base_name}_filtered.csv")
    df_filtered.to_csv(filtered_path, index=False)
    print(f"Filtered data saved to {filtered_path}")

    # Step 5: Format as .01a-style with header
    formatted_data = []
    for entry_counter, row in enumerate(df_filtered.itertuples(index=False)):
        mv = f"{row.mV:.3f}"
        cumulative_volume = f"{row.volume:.3f}"
        entry_str = f" {entry_counter}, {cumulative_volume}, {mv}\n"
        formatted_data.append(entry_str)

    output_lines = header_lines + formatted_data

    # Step 6: Save final .01a file
    formatted_output_path = os.path.join(directory, f"{base_name}.01a")
    with open(formatted_output_path, "w") as out_file:
        out_file.writelines(output_lines)

    print(f"Formatted .01a file saved to {formatted_output_path}")


Filtered data saved to /Users/cameronrichardson/Documents/Thesis/Experiments/titration data/10C/Ca/csv/~$S0618C41N01_filtered.csv
Formatted .01a file saved to /Users/cameronrichardson/Documents/Thesis/Experiments/titration data/10C/Ca/csv/~$S0618C41N01.01a
Filtered data saved to /Users/cameronrichardson/Documents/Thesis/Experiments/titration data/10C/Ca/csv/~$S0618C41N02_filtered.csv
Formatted .01a file saved to /Users/cameronrichardson/Documents/Thesis/Experiments/titration data/10C/Ca/csv/~$S0618C41N02.01a
